<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>
Figure 1.1.a. Jupyter environment + Python notebooks

### Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

Step 1.1

In [1]:
# Predefined variable for title and version
version = "site_rf_14052025"# Replace this with the variable you want to use

In [2]:
from pystac_client import Client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import rasterio as rio
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import folium
import utils
# from utils import apply_masks
from utils import scale
from utils import calculate_band_indices
from utils import load_data
# from utils import mask_deeps_stumpf
# from utils import mask_deeps_ln_bg
# from utils import mask_deeps
# from utils import do_prediction 
# from utils import mask_and_scale
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
import odc.geo.xr
import rioxarray
import matplotlib.pyplot as plt
import joblib
from shapely.geometry import box
import skimage.feature
from skimage.feature import graycomatrix, graycoprops
from skimage import data

Define catalogue

In [3]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [4]:
gdf = gpd.read_file("training-data/Alluvium_png_atolau_05022025.geojson")
gdf = gdf.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = gdf.total_bounds
bbox = [min_lon, min_lat, max_lon, max_lat]

In [5]:
# min_lon = 160.08855
# min_lat = -9.12915
# max_lon = 160.17137
# max_lat = -9.08003

# bbox = [min_lon, min_lat, max_lon, max_lat]

In [6]:
datetime="2024"
items = list(client.search(collections=["dep_s2_geomad"], datetime=datetime, bbox=bbox).items())

In [7]:
from pystac import Collection

In [8]:
collection = Collection.from_file("https://stac.digitalearthpacific.org/collections/dep_s2_geomad")

In [9]:
data = load(
        items,
        measurements=[
            "nir", "red", "blue", "green", "emad", "smad", 
            "bcmad", "count", "green", "nir08", 
            "nir09", "swir16", "swir22", "coastal",
            "rededge1", "rededge2", "rededge3", 
        ],
        bbox=bbox,
        chunks={"x": 2048, "y": 2048},
        groupby="solar_day",
    )

In [10]:
# dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')
# configure_s3_access(cloud_defaults=True, requester_pays=True)

In [11]:
scaled = (data.where(data != 0) * 0.0001).clip(0, 1)

In [12]:
mndwi = (scaled["green"]-scaled["swir16"])/(scaled["green"]+scaled["swir16"])
## Moderate land mask
# mndwi_land_mask = mndwi > 0
mndwi_land_mask = mndwi > -1.5
clipped_ds = scaled.where(mndwi_land_mask)

In [13]:
ndti = (clipped_ds["red"]-clipped_ds["green"])/(clipped_ds["red"]+clipped_ds["green"])
ndti_mask = ndti < 0.2
clipped_ds = clipped_ds.where(ndti_mask)

In [14]:
nir = clipped_ds['nir']

# Moderate land mask
# nir_mask = nir < 0.085

# Conservative land mask
nir_mask = nir < 0.8
clipped_ds = clipped_ds.where(nir_mask)

In [15]:
# Incorporate other band ratios and indices
cai = (clipped_ds["coastal"]-clipped_ds["blue"])/( clipped_ds["coastal"]+ clipped_ds["blue"]) #coastal aerosol index
ndvi = (clipped_ds["nir"]-clipped_ds["red"])/( clipped_ds["nir"]+ clipped_ds["red"]) #vegetation index (NDVI)
evi = ((2.5*clipped_ds["nir"]-clipped_ds["red"])/(clipped_ds["nir"]+(6*clipped_ds["red"])-(7.5*clipped_ds["blue"])+1)) # enhanced vegetation index
savi = (clipped_ds["nir"]-clipped_ds["red"])/(clipped_ds["nir"]+clipped_ds["red"]) # soil adjusted vegetation index
ndwi = (clipped_ds["green"]-clipped_ds["nir"])/(clipped_ds["green"]+clipped_ds["nir"]+0.428)*(1+0.428) #water index (NDWI)
b_g = (clipped_ds["blue"])/(clipped_ds["green"]) #blue to green ratio
b_r = (clipped_ds["blue"])/(clipped_ds["red"]) #blue to red ratio
mci = (clipped_ds["nir"])/(clipped_ds["rededge1"]) # max chlorophlyll index (MCI)
ndci = (clipped_ds["rededge1"]-clipped_ds["red"])/(clipped_ds["rededge1"]+clipped_ds["red"]) # normalised difference chlorophyll index (NDCI)


In [16]:
clipped_ds['cai'] = cai
clipped_ds['ndvi'] = ndvi
clipped_ds['evi'] = evi
clipped_ds['savi'] = savi
clipped_ds['ndwi'] = ndwi
clipped_ds['mndwi'] = mndwi
clipped_ds['ndti'] = ndti
clipped_ds['b_g'] = b_g
clipped_ds['b_r'] = b_r
clipped_ds['mci'] = mci
clipped_ds['ndci'] = ndci

In [17]:
# Natural log of blue/green
clipped_ds["ln_bg"] = np.log(clipped_ds.blue / clipped_ds.green)
bg = clipped_ds["ln_bg"]
# conservative deep sea mask
mask_bg = bg < 0.0000001

In [18]:
all_masks = (mndwi_land_mask+ndti_mask+nir_mask+mask_bg)

### GLCM texture analysis

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [19]:
#texture_image = clipped_ds.to_dataarray()

In [20]:
#texture_image

### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [21]:
# Define training data
gdf = gpd.read_file("training-data/Alluvium_png_atolau_05022025.geojson")
gdf = gdf.to_crs("EPSG:4326")
# gdf.explore(column="cc_id", legend=True)
clipped_ds

<xarray.Dataset> Size: 246MB
Dimensions:      (time: 1, y: 609, x: 3610)
Coordinates:
  * y            (y) float64 5kB -1.145e+06 -1.145e+06 ... -1.151e+06 -1.151e+06
  * x            (x) float64 29kB 3.782e+04 3.784e+04 ... 7.39e+04 7.392e+04
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/28)
    nir          (time, y, x) float32 9MB dask.array<chunksize=(1, 609, 2048), meta=np.ndarray>
    red          (time, y, x) float32 9MB dask.array<chunksize=(1, 609, 2048), meta=np.ndarray>
    blue         (time, y, x) float32 9MB dask.array<chunksize=(1, 609, 2048), meta=np.ndarray>
    green        (time, y, x) float32 9MB dask.array<chunksize=(1, 609, 2048), meta=np.ndarray>
    emad         (time, y, x) float32 9MB dask.array<chunksize=(1, 609, 2048), meta=np.ndarray>
    smad         (time, y, x) float32 9MB dask.array<chunksize=(1, 609, 2048), meta=np.ndarray>
    ...           ...
    ndti         (time, y, x) float32 9MB dask.array<chunksize=(1, 609, 2048), meta=np.ndarray>
    b_g          (time, y, x) float32 9MB dask.array<chunksize=(1, 609, 2048), meta=np.ndarray>
    b_r          (time, y, x) float32 9MB dask.array<chunksize=(1, 609, 2048), meta=np.ndarray>
    mci          (time, y, x) float32 9MB dask.array<chunksize=(1, 609, 2048), meta=np.ndarray>
    ndci         (time, y, x) float32 9MB dask.array<chunksize=(1, 609, 2048), meta=np.ndarray>
    ln_bg        (time, y, x) float32 9MB dask.array<chunksize=(1, 609, 2048), meta=np.ndarray>

In [22]:
postcard = clipped_ds#.where(all_masks)
# postcard =clipped_ds.to_array(dim="band")
# First transform the training points to the same CRS as the data
training = gdf.to_crs(postcard.odc.geobox.crs)

In [23]:
print(training.columns)
training=training.drop(columns=['date', 'uuid'])

Index(['uuid', 'cc_id', 'observed', 'date', 'geometry'], dtype='object')


In [24]:
training

,cc_id,observed,geometry
0,4,seagrass,POINT (71804.113 -1150384.928)
1,10,land,POINT (38054.247 -1149649.551)
2,10,land,POINT (45707.276 -1144915.566)
3,9,mangrove,POINT (45248.813 -1145020.018)
4,9,mangrove,POINT (45154.984 -1145080.51)
...,...,...,...
474,10,land,POINT (38380.485 -1150749.059)
475,9,mangrove,POINT (38384.599 -1150759.335)
476,9,mangrove,POINT (38387.497 -1150771.348)
477,10,land,POINT (38409.64 -1150896.344)


In [25]:
training_da = training.assign(x=training.geometry.x, y=training.geometry.y).to_xarray()

In [26]:
training_values = (
    postcard.sel(training_da[["x", "y"]], method="nearest").squeeze().compute().to_pandas()
)
training_values

,nir,red,blue,green,emad,smad,bcmad,count,nir08,nir09,...,evi,savi,ndwi,mndwi,ndti,b_g,b_r,mci,ndci,ln_bg
index,,,,,,,,,,,,,,,,,,,,,
0,0.0513,0.0564,0.0663,0.0711,0.093868,3.085601e-06,0.000026,0.0017,0.0442,0.1120,...,0.080509,-0.047354,0.051371,0.367308,-0.115294,0.932489,1.175532,0.725601,0.112510,-0.069897
1,0.2878,0.0465,0.0450,0.0688,0.048979,2.053738e-07,0.000004,0.0013,0.2588,0.2203,...,0.547466,0.721807,-0.398588,-0.280335,-0.193408,0.654070,0.967742,3.128261,0.328520,-0.424541
2,0.3655,0.0340,0.0392,0.0656,0.120779,2.141744e-07,0.000007,0.0008,0.3583,0.2738,...,0.689730,0.829787,-0.498495,-0.401187,-0.317269,0.597561,1.152941,3.803330,0.477325,-0.514899
3,0.2328,0.0380,0.0413,0.0576,0.079367,4.400849e-07,0.000006,0.0007,0.2737,0.2989,...,0.472612,0.719350,-0.348254,-0.320755,-0.205021,0.717014,1.086842,2.645454,0.396825,-0.332660
4,0.0815,0.0600,0.0808,0.0712,0.198120,7.706034e-06,0.000040,0.0008,0.0917,0.2440,...,0.172053,0.151943,-0.025329,0.073906,-0.085366,1.134831,1.346667,1.165951,0.076212,0.126484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,0.3426,0.0354,0.0411,0.0684,0.042198,5.122423e-08,0.000003,0.0014,0.3713,0.3374,...,0.658592,0.812698,-0.466696,-0.344514,-0.317919,0.600877,1.161017,3.405567,0.479412,-0.509365
475,0.3411,0.0334,0.0401,0.0664,0.042129,6.314516e-08,0.000003,0.0014,0.3714,0.3373,...,0.660367,0.821629,-0.469505,-0.358454,-0.330661,0.603916,1.200599,3.387289,0.501864,-0.504321
476,0.3101,0.0348,0.0407,0.0647,0.057117,7.084608e-08,0.000004,0.0014,0.3506,0.3285,...,0.610102,0.798202,-0.436511,-0.359406,-0.300503,0.629057,1.169540,3.233577,0.467483,-0.463533


In [27]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["cc_id"], training_values], axis=1)
# Drop rows where there was no data available
training_array = training_array.dropna()
# Preview our resulting training array
training_array.head()

,cc_id,nir,red,blue,green,emad,smad,bcmad,count,nir08,...,evi,savi,ndwi,mndwi,ndti,b_g,b_r,mci,ndci,ln_bg
0,4,0.0513,0.0564,0.0663,0.0711,0.093868,3.085601e-06,0.000026,0.0017,0.0442,...,0.080509,-0.047354,0.051371,0.367308,-0.115294,0.932489,1.175532,0.725601,0.112510,-0.069897
1,10,0.2878,0.0465,0.0450,0.0688,0.048979,2.053738e-07,0.000004,0.0013,0.2588,...,0.547466,0.721807,-0.398588,-0.280335,-0.193408,0.654070,0.967742,3.128261,0.328520,-0.424541
2,10,0.3655,0.0340,0.0392,0.0656,0.120779,2.141744e-07,0.000007,0.0008,0.3583,...,0.689730,0.829787,-0.498495,-0.401187,-0.317269,0.597561,1.152941,3.803330,0.477325,-0.514899
3,9,0.2328,0.0380,0.0413,0.0576,0.079367,4.400849e-07,0.000006,0.0007,0.2737,...,0.472612,0.719350,-0.348254,-0.320755,-0.205021,0.717014,1.086842,2.645454,0.396825,-0.332660
4,9,0.0815,0.0600,0.0808,0.0712,0.198120,7.706034e-06,0.000040,0.0008,0.0917,...,0.172053,0.151943,-0.025329,0.073906,-0.085366,1.134831,1.346667,1.165951,0.076212,0.126484


In [28]:
# Setup soectral_features_df



In [29]:
# training_features = []
# for image in texture_image:  # Assume texture_image is a list of 2D arrays
#     # Ensure each image is 2D
#     if image.ndim == 3:
#         image_blue = image[:, :, 2]  # Extract the first band

# image_blue = ((image_blue - image_blue.min()) / (image_blue.max() - image_blue.min()) * 255).astype('uint8')


# # Function to calculate GLCM metrics
# def compute_glcm_features(image_blue, distances, angles, levels):
#     glcm = graycomatrix(image_blue, distances=distances, angles=angles, levels=levels, symmetric=True, normed=True)
#     features = {
#         'contrast': graycoprops(glcm, 'contrast')[0, 0],
#         'homogeneity': graycoprops(glcm, 'homogeneity')[0, 0],
#         'energy': graycoprops(glcm, 'energy')[0, 0],
#         'correlation': graycoprops(glcm, 'correlation')[0, 0]
#     }
#     return features

# # Example: Calculate GLCM metrics for each image/region
# training_features = []
# for image in image_blue:  # training_images is a list of 2D arrays
#     features = compute_glcm_features(image_blue, distances=[1], angles=[0], levels=256)
#     training_features.append(features)

# # Convert features into a DataFrame or array
# import pandas as pd
# training_features_df = pd.DataFrame(training_features)



In [30]:
# contrast = features.['contrast'] 

In [31]:
# def overlay_texture_feature(clipped_ds, training_features, alpha=0.6):
#     plt.figure(figsize=(8, 8))
#     plt.imshow(original_image, cmap='gray')
#     plt.imshow(feature_array, cmap='viridis', alpha=alpha)  # Overlay with transparency
#     plt.colorbar(label='Texture Feature')
#     plt.title("Texture Feature Overlay")
#     plt.axis('off')
#     plt.show()

# # Example: Overlay contrast on the original image
# overlay_texture_feature(image_blue, contrast_array)

In [32]:
# Combine GLCM features with spectral features
#combined_features = pd.concat([spectral_features_df, training_features_df], axis=1)



# # Train a Random Forest Classifier
# X_train, X_test, y_train, y_test = train_test_split(combined_features, labels, test_size=0.2, random_state=42)
# rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_model.fit(X_train, y_train)

# # Make predictions
# predictions = rf_model.predict(X_test)

### Notebook 2 - Train Random Forest Machine Learning (ML) Model

Combine the csv geodataframes from notebook 1 into a single csv to train the machine learning model

Step 2.1. Concatenating all postcard dataframes

In [33]:
postcard_df = training_array
postcard_df.columns

Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'count', 'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
       'rededge2', 'rededge3', 'y', 'x', 'spatial_ref', 'time', 'cai', 'ndvi',
       'evi', 'savi', 'ndwi', 'mndwi', 'ndti', 'b_g', 'b_r', 'mci', 'ndci',
       'ln_bg'],
      dtype='object')

In [34]:
postcard_df.columns

Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'count', 'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
       'rededge2', 'rededge3', 'y', 'x', 'spatial_ref', 'time', 'cai', 'ndvi',
       'evi', 'savi', 'ndwi', 'mndwi', 'ndti', 'b_g', 'b_r', 'mci', 'ndci',
       'ln_bg'],
      dtype='object')

In [35]:
# postcard_df.columns
# postcard_df = postcard_df.drop(columns=["spatial_ref", "time", "field_1", "y", "x"])
# postcard_df = postcard_df.drop(columns=["field_1"])
postcard_df = postcard_df.drop(columns=["y", "x", "spatial_ref"])
postcard_df = postcard_df.drop(columns=["time"])
postcard_df

,cc_id,nir,red,blue,green,emad,smad,bcmad,count,nir08,...,evi,savi,ndwi,mndwi,ndti,b_g,b_r,mci,ndci,ln_bg
0,4,0.0513,0.0564,0.0663,0.0711,0.093868,3.085601e-06,0.000026,0.0017,0.0442,...,0.080509,-0.047354,0.051371,0.367308,-0.115294,0.932489,1.175532,0.725601,0.112510,-0.069897
1,10,0.2878,0.0465,0.0450,0.0688,0.048979,2.053738e-07,0.000004,0.0013,0.2588,...,0.547466,0.721807,-0.398588,-0.280335,-0.193408,0.654070,0.967742,3.128261,0.328520,-0.424541
2,10,0.3655,0.0340,0.0392,0.0656,0.120779,2.141744e-07,0.000007,0.0008,0.3583,...,0.689730,0.829787,-0.498495,-0.401187,-0.317269,0.597561,1.152941,3.803330,0.477325,-0.514899
3,9,0.2328,0.0380,0.0413,0.0576,0.079367,4.400849e-07,0.000006,0.0007,0.2737,...,0.472612,0.719350,-0.348254,-0.320755,-0.205021,0.717014,1.086842,2.645454,0.396825,-0.332660
4,9,0.0815,0.0600,0.0808,0.0712,0.198120,7.706034e-06,0.000040,0.0008,0.0917,...,0.172053,0.151943,-0.025329,0.073906,-0.085366,1.134831,1.346667,1.165951,0.076212,0.126484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,10,0.3426,0.0354,0.0411,0.0684,0.042198,5.122423e-08,0.000003,0.0014,0.3713,...,0.658592,0.812698,-0.466696,-0.344514,-0.317919,0.600877,1.161017,3.405567,0.479412,-0.509365
475,9,0.3411,0.0334,0.0401,0.0664,0.042129,6.314516e-08,0.000003,0.0014,0.3714,...,0.660367,0.821629,-0.469505,-0.358454,-0.330661,0.603916,1.200599,3.387289,0.501864,-0.504321
476,9,0.3101,0.0348,0.0407,0.0647,0.057117,7.084608e-08,0.000004,0.0014,0.3506,...,0.610102,0.798202,-0.436511,-0.359406,-0.300503,0.629057,1.169540,3.233577,0.467483,-0.463533
477,10,0.3272,0.0322,0.0380,0.0620,0.051636,9.370744e-08,0.000003,0.0016,0.3566,...,0.636069,0.820812,-0.463418,-0.380000,-0.316348,0.612903,1.180124,3.331975,0.506135,-0.489548


In [36]:
len(postcard_df.columns)

29

In [37]:
joined_df = pd.read_csv("training-data/joined_training_data_11042025_test.csv")
joined_df.columns

Index(['Unnamed: 0', 'cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad',
       'bcmad', 'count', 'nir08', 'nir09', 'swir16', 'swir22', 'coastal',
       'rededge1', 'rededge2', 'rededge3', 'cai', 'ndvi', 'ndwi', 'mndwi',
       'ndti', 'b_g', 'b_r', 'mci', 'ndci', 'y', 'x'],
      dtype='object')

In [38]:
joined_df = joined_df.drop(columns=["Unnamed: 0"])
joined_df.columns

Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'count', 'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
       'rededge2', 'rededge3', 'cai', 'ndvi', 'ndwi', 'mndwi', 'ndti', 'b_g',
       'b_r', 'mci', 'ndci', 'y', 'x'],
      dtype='object')

In [39]:
correlation_matrix = joined_df.corr()

# Display the matrix
print(correlation_matrix)

             cc_id       nir       red      blue     green      emad  \
cc_id     1.000000  0.251314  0.191743  0.196799  0.198334  0.338373   
nir       0.251314  1.000000  0.719140  0.435462  0.507866  0.800955   
red       0.191743  0.719140  1.000000  0.857162  0.928126  0.621408   
blue      0.196799  0.435462  0.857162  1.000000  0.963797  0.541867   
green     0.198334  0.507866  0.928126  0.963797  1.000000  0.517382   
emad      0.338373  0.800955  0.621408  0.541867  0.517382  1.000000   
smad     -0.263010 -0.525267 -0.649557 -0.609282 -0.636978 -0.412608   
bcmad    -0.000468 -0.232421 -0.498055 -0.367957 -0.496034  0.177283   
count    -0.109710 -0.349274 -0.492707 -0.316153 -0.467422 -0.111915   
nir08     0.281652  0.942935  0.668800  0.433399  0.486698  0.830376   
nir09     0.108424  0.663542  0.431132  0.242825  0.274995  0.566750   
swir16    0.383256  0.793337  0.601835  0.508957  0.507362  0.865964   
swir22    0.408891  0.746193  0.569726  0.519196  0.499119  0.87

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

In [ ]:
threshold = 0.8
high_corr = correlation_matrix[(correlation_matrix > threshold) & (correlation_matrix != 1.0)]
print(high_corr)

In [ ]:
high_corr.to_csv(f"{version}_high_correlation_matrix.csv")

Step 2.2. Train the ML model

In [ ]:
# The classes are the first column
classes = np.array(postcard_df)[:, 0]

# The observation data is everything after the second column
observations = np.array(postcard_df)[:, 1:]

# Create a model...
classifier = RandomForestClassifier()

# ...and fit it to the data
model = classifier.fit(observations, classes)

In [ ]:
# Dynamically create the filename with f-string
file_path = f"models/{version}-test.model"

# Save the model
joblib.dump(model, file_path)

In [ ]:
postcard_params = postcard_df.columns[1:]
print(postcard_params)

In [ ]:
importances = classifier.feature_importances_

# Create the DataFrame
data = {
    'param': postcard_params,  # Use the list directly
    'importance': importances,  # Use the list directly
}
importance_df = pd.DataFrame(data)

importance_df["%"] = (importance_df["importance"]*100)

sorted_df = importance_df.sort_values(by='importance', ascending=False)
sorted_df.to_csv(f"{version}importance_df.csv")

In [ ]:
sorted_df
sorted_df.to_csv('sorted.csv', index=False)

In [ ]:
postcard


In [ ]:
# Convert to a stacked array of observations
# stacked_arrays = stacked_arrays.squeeze()
stacked_arrays = postcard.squeeze()#.stack(dims=["y", "x"])#.transpose()
stacked_arrays = stacked_arrays.to_dataarray()

In [ ]:
stacked_arrays

In [ ]:
stacked_arrays_2d = stacked_arrays.stack(new_dim=("y", "x")) 

In [ ]:
reordered_data_array = stacked_arrays_2d.transpose('new_dim', 'variable')

In [ ]:
# stacked_arrays = stacked_arrays.reshape(-1, 26)
stacked_arrays_2d.shape


In [ ]:
# Convert to a stacked array of observations
# stacked_arrays_2d = stacked_arrays_2d.to_array().stack(dims=["y", "x"])

# Replace any infinities with NaN
stacked_arrays_2d = stacked_arrays_2d.where(stacked_arrays_2d != float("inf"))
stacked_arrays_2d = stacked_arrays_2d.where(stacked_arrays_2d != float("-inf"))

# Replace any NaN values with 0
df = stacked_arrays_2d.squeeze().fillna(0).transpose().to_pandas()

# Remove the all-zero rows
zero_mask = (df == 0).all(axis=1)  # Creates a boolean Series
non_zero_df = df.loc[~zero_mask]  # Filters out all-zero rows

# Create a new array to hold the predictions
full_pred = pd.Series(np.nan, index=df.index)

reordered_data_array = stacked_arrays_2d.transpose('new_dim', 'variable')

In [ ]:
# Predict the classes

predicted = model.predict(reordered_data_array)


In [ ]:

# Reshape back to the original 2D array
reordered_data_array = predicted.reshape(len(postcard.y), len(postcard.x))

# Convert to an xarray again, because it's easier to work with
predicted_da = xr.DataArray(
    reordered_data_array, coords={"y": postcard.y, "x": postcard.x}, dims=["y", "x"]
)

In [ ]:
print(predicted_da.dtype)  # Check the dtype of your DataArray
predicted_da = predicted_da.astype('float32')  # Convert to float32

# Check for NaN values
if np.isnan(predicted_da).any():
    print("NaN values found in the data")
    # Handle NaN values, e.g. by filling them
    predicted_da = predicted_da.fillna(0)  # Replace NaN with 0 or appropriate value

In [ ]:
predicted_da

In [ ]:
from matplotlib import colors

classes = [
    [1, "sediment", "#8c8c8c"],
    [2, "sand", "#fedd24"],
    [3, "rubble", "#f8ffb4"],
    [4, "seagrass", "#6df7dc"],
    [5, "seaweed", "#b9df6f"],
    [6, "coral", "#a011c3"],
    [7, "rock", "#804600"],
    [8, "deeps", "#011b61"],
    [9, "mangrove", "#086a39"],
    [10, "land", "#ffffff"],
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map = colors.ListedColormap(color_list)
bounds = values_list + [14]
norm = colors.BoundaryNorm(bounds, c_map.N)

predicted_da.plot.imshow(cmap=c_map, norm=norm, size=10)

In [ ]:
predicted_da.odc.explore()

In [ ]:
# issue where not all masks are being included - only land but not surf / also strange effect on side
predicted_da = predicted_da.where(all_masks)

In [ ]:
predicted_da.odc.explore(cmap=c_map)

In [ ]:
# issue where not all masks are being included - only land but not surf / also strange effect on side
predicted_da = predicted_da.where(bg < 0.2)
predicted_da.odc.explore(cmap=c_map)

In [65]:
predicted_da.odc.write_cog(f"{version}.tiff", overwrite=True)

PosixPath('malekula_joined_model_fi.tiff')